In [ ]:
# !pip install polars
# !pip install kaggle
# !pip install matplotlib

In [5]:
# !kaggle datasets download -d mrmorj/alfabattle-20
# !unzip -qq alfabattle-20.zip

100%|██████████████████████████████████████| 4.37G/4.37G [14:22<00:00, 6.04MB/s]
100%|██████████████████████████████████████| 4.37G/4.37G [14:22<00:00, 5.44MB/s]


In [1]:
import polars as pl
import matplotlib.pyplot as plt

In [2]:
%%time
df = pl.read_parquet('alfabattle2_train_transactions_contest/train_transactions_contest/*.parquet')
print(f'Estimated size: {df.estimated_size('gb')}')

Estimated size: 24.180120080709457
CPU times: user 35.3 s, sys: 27.9 s, total: 1min 3s
Wall time: 26.4 s


In [3]:
df.head()

app_id,amnt,currency,operation_kind,card_type,operation_type,operation_type_group,ecommerce_flag,payment_system,income_flag,mcc,country,city,mcc_category,day_of_week,hour,days_before,weekofyear,hour_diff,transaction_number,__index_level_0__
i32,f64,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i64,i32,i64
0,0.465425,1,4,98,4,2,3,7,3,2,1,37,2,4,19,351,34,-1,1,0
0,0.0,1,2,98,7,1,3,7,3,2,1,49,2,4,20,351,34,0,2,1
0,0.521152,1,2,98,3,1,3,7,3,2,1,37,2,4,20,351,34,0,3,2
0,0.356078,1,1,5,2,1,3,7,3,10,1,49,7,2,0,348,34,52,4,3
0,0.0,1,2,98,7,1,3,7,3,2,1,49,2,4,16,337,53,280,5,4


In [5]:
# cast several column to reduce df size
df = df.with_columns(
    pl.col('currency').cast(pl.Int8),
    pl.col('operation_type').cast(pl.Int8),
    pl.col('income_flag').cast(pl.Int8),
    pl.col('payment_system').cast(pl.Int8),
    pl.col('ecommerce_flag').cast(pl.Int8),
    pl.col('mcc_category').cast(pl.Int8),
    pl.col('country').cast(pl.Int8),
    pl.col('mcc').cast(pl.Int8),
    pl.col('day_of_week').cast(pl.Int8),
    pl.col('hour').cast(pl.Int8),
    pl.col('weekofyear').cast(pl.Int8),
    pl.col('days_before').cast(pl.Int16),
    pl.col('city').cast(pl.Int16),
    pl.col('card_type').cast(pl.Int16),
    pl.col('operation_kind').cast(pl.Int16), 
    pl.col('amnt').cast(pl.Float32)
    )
print(f'Estimated size after casting: {df.estimated_size('gb')}')

Estimated size after casting: 12.8456887928769


In [6]:
# read target 
users = pl.read_csv(
    'alfabattle2_train_target.csv',
    dtypes={'app_id' : pl.Int32}
    )

In [7]:
%%time
# join tables
merged = df.join(other=users, on='app_id')

CPU times: user 7.89 s, sys: 18.5 s, total: 26.4 s
Wall time: 15.7 s


In [8]:
print(f'Estimated size after join operation: {merged.estimated_size('gb')}')

Estimated size after join operation: 16.875708806328475


In [9]:
merged.head()

app_id,amnt,currency,operation_kind,card_type,operation_type,operation_type_group,ecommerce_flag,payment_system,income_flag,mcc,country,city,mcc_category,day_of_week,hour,days_before,weekofyear,hour_diff,transaction_number,__index_level_0__,product,flag
i32,f32,i8,i16,i16,i8,i32,i8,i8,i8,i8,i8,i16,i8,i8,i8,i16,i8,i64,i32,i64,i64,i64
0,0.465425,1,4,98,4,2,3,7,3,2,1,37,2,4,19,351,34,-1,1,0,3,0
0,0.0,1,2,98,7,1,3,7,3,2,1,49,2,4,20,351,34,0,2,1,3,0
0,0.521152,1,2,98,3,1,3,7,3,2,1,37,2,4,20,351,34,0,3,2,3,0
0,0.356078,1,1,5,2,1,3,7,3,10,1,49,7,2,0,348,34,52,4,3,3,0
0,0.0,1,2,98,7,1,3,7,3,2,1,49,2,4,16,337,53,280,5,4,3,0
